#As the first step, Data preprocessing is done, which includes loading the dataset, and then splitting it into training and testing sets with a percentage of 80% to 20%. Finally the data is scaled to a common scale using minmax scaler which ranges the data from 0 to 1

In [1]:
import pandas as pd
#load the dataset
data=pd.read_csv('dataset/spambase.data',header=None)

In [2]:
#splitting the dataset into training and testing
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)



In [4]:
#scaling the data to improve performance
from sklearn.preprocessing import MinMaxScaler, StandardScaler


scaler = MinMaxScaler()

X_train_= scaler.fit_transform(X_train)
X_test_=scaler.transform(X_test)



#KNN model

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Create a KNN classifier
knn = KNeighborsClassifier()

# Define the hyperparameter grid
param_grid = {'n_neighbors': np.arange(1, 50),
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}

# Create a grid search object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train_, y_train)

# Print the best hyperparameters and accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)


Best hyperparameters: {'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Best accuracy: 0.923913043478261
